In [1]:
import polars as pl
import optuna
import xgboost as xgb
import numpy as np

from src.util.constants import DATA_PATH, META_MODEL_PERFORMANCE, FIXED_XGB_PARAMETERS
from src.util.common import load_from_pickle, mean_grouped_spearman_correlation, save_as_pickle

/Users/jonas-data-science/Library/Caches/pypoetry/virtualenvs/nmr-DUJvlELt-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
selected_features = load_from_pickle(DATA_PATH / 'results/selected_features.pkl')
required_columns = ['era', 'target'] + selected_features

df_train_list = []
df_validate_list = []
for fold in range(2):
    df_train_fold = pl.read_parquet(f"{DATA_PATH}/folds/df_train_{fold}.parquet")
    df_validate_fold = pl.read_parquet(f"{DATA_PATH}/folds/df_validate_{fold}.parquet")

    df_train_fold = df_train_fold.select(required_columns)
    df_validate_fold = df_validate_fold.select(required_columns)

    df_train_list.append(df_train_fold)
    df_validate_list.append(df_validate_fold)
    del df_train_fold, df_validate_fold


df_meta_model = pl.read_parquet(f'{DATA_PATH}/folds/df_meta_model.parquet')
df_meta_model = df_meta_model.select(required_columns + ['numerai_meta_model'])

NameError: name 'load_from_pickle' is not defined

In [4]:
def performance_approximation(model: xgb.Booster, fold: int) -> tuple[float, float, float, float]:
    df_validate_fold = df_validate_list[fold]

    dmatrix_validate = xgb.DMatrix(df_validate_fold[selected_features].to_numpy())
    corr = mean_grouped_spearman_correlation(
        pl.Series(model.predict(dmatrix_validate)),
        df_validate_fold['target'],
        df_validate_fold['era']
    )

    dmatrix_meta = xgb.DMatrix(df_meta_model[selected_features].to_numpy())
    df_mm_corr = pl.DataFrame({
        'prediction': pl.Series(model.predict(dmatrix_meta)),
        'numerai_meta_model': df_meta_model['numerai_meta_model'],
        'era': df_meta_model['era']
    })
    corr_w_mm = df_mm_corr.select(
        pl.corr("prediction", "numerai_meta_model", method="spearman")
        .over('era', mapping_strategy='explode')
    ).mean().item()

    mmc_approximation = corr - corr_w_mm * META_MODEL_PERFORMANCE[fold]
    performance = .75 * corr + 2.25 * mmc_approximation

    return performance, corr, corr_w_mm, mmc_approximation

In [5]:
def objective(trial: optuna.trial.Trial) -> float:
    # we use these exponents to create a log scale that includes zero
    min_child_weight_exponent = trial.suggest_int("min_child_weight_exponent", 0, 10)
    alpha_exponent = trial.suggest_int("alpha_exponent", 0, 8)  # L1 regularization
    lambda_exponent = trial.suggest_int("lambda_exponent", 0, 12)  # L2 regularization
    parameter_sample = {
        "learning_rate": trial.suggest_float("learning_rate", .001, .2, log=True),
        "num_boost_round": trial.suggest_int("num_boost_round", 10, 5000),
        'max_bin': trial.suggest_int('max_bin', 2 ** 3, 2 ** 10, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 12),
        "subsample": trial.suggest_float("subsample", .1, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", .1, 1.0),
        "min_child_weight": (10 ** min_child_weight_exponent - 1) / 10 ** 5,
        "alpha": (10 ** alpha_exponent - 1) / 10 ** 5,
        "lambda": (10 ** lambda_exponent - 1) / 10 ** 5
    }

    # extract num_boost_round (not a param in xgb.train params dict)
    num_boost_round = parameter_sample.pop('num_boost_round')

    parameters = {
        **FIXED_XGB_PARAMETERS,
        **parameter_sample
    }

    result_performance = []

    for fold in range(2):
        dtrain = xgb.DMatrix(
            df_train_list[fold][selected_features].to_numpy(),
            label=df_train_list[fold]['target'].to_numpy()
        )

        model = xgb.train(
            params=parameters,
            dtrain=dtrain,
            num_boost_round=num_boost_round
        )

        performance, corr, corr_w_mm, mmc_approximation = performance_approximation(model, fold)
        trial.set_user_attr(f"performance_{fold}", performance)
        trial.set_user_attr(f"correlation_{fold}", corr)
        trial.set_user_attr(f"correlation_with_meta_model_{fold}", corr_w_mm)
        trial.set_user_attr(f"mmc_approximation_{fold}", mmc_approximation)
        print(f'Results for fold {fold}: Performance {performance:.5f}, correlation: {corr:.5f}, correlation with meta model: {corr_w_mm:.5f}, MMC (approx.): {mmc_approximation:.5f}')

        # stop trial if it does not show promise (by default, median pruner is used)
        trial.report(performance, step=fold)
        if fold == 0 and trial.number > 10 and trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        result_performance.append(performance)

    return np.mean(result_performance)

In [6]:
study = optuna.create_study(direction="maximize")

[I 2026-01-26 09:42:43,126] A new study created in memory with name: no-name-b88ac76f-ecc1-4db5-a630-76cd0b891cba


In [ ]:
# cumulative performance 50 runs:
# best performance: 0.0500
# runtime: 24:10

In [7]:
study.optimize(objective, n_trials=8, show_progress_bar=True)

  0%|          | 0/8 [00:00<?, ?it/s]

Results for fold 0: Performance 0.00512, correlation: 0.01268, correlation with meta model: 0.33250, MMC (approx.): -0.00195
Results for fold 1: Performance 0.02019, correlation: 0.01511, correlation with meta model: 0.30179, MMC (approx.): 0.00394


Best trial: 0. Best value: 0.0126558:  12%|█▎        | 1/8 [30:35<3:34:07, 1835.33s/it]

[I 2026-01-26 10:13:37,317] Trial 0 finished with value: 0.012655762868013506 and parameters: {'min_child_weight_exponent': 6, 'alpha_exponent': 5, 'lambda_exponent': 12, 'learning_rate': 0.003273004290569747, 'num_boost_round': 3089, 'max_bin': 36, 'max_depth': 5, 'subsample': 0.693541712992084, 'colsample_bytree': 0.44060310421904575}. Best is trial 0 with value: 0.012655762868013506.
Results for fold 0: Performance 0.04083, correlation: 0.03303, correlation with meta model: 0.58833, MMC (approx.): 0.00714
Results for fold 1: Performance 0.03073, correlation: 0.02633, correlation with meta model: 0.57969, MMC (approx.): 0.00488


Best trial: 1. Best value: 0.0357798:  25%|██▌       | 2/8 [1:22:20<4:18:14, 2582.48s/it]

...

[I 2026-01-26 13:36:09,144] Trial 5 finished with value: 0.023347574682710214 and parameters: {'min_child_weight_exponent': 9, 'alpha_exponent': 1, 'lambda_exponent': 8, 'learning_rate': 0.0010896229108451829, 'num_boost_round': 4224, 'max_bin': 19, 'max_depth': 4, 'subsample': 0.95271970837706, 'colsample_bytree': 0.4114311515180548}. Best is trial 1 with value: 0.03577978221602787.
Results for fold 0: Performance 0.05282, correlation: 0.03703, correlation with meta model: 0.58852, MMC (approx.): 0.01113
Results for fold 1: Performance 0.03711, correlation: 0.02878, correlation with meta model: 0.59148, MMC (approx.): 0.00690


Best trial: 6. Best value: 0.0449676:  88%|████████▊ | 7/8 [4:06:43<26:34, 1594.98s/it]  

[I 2026-01-26 13:49:45,548] Trial 6 finished with value: 0.04496761808814566 and parameters: {'min_child_weight_exponent': 8, 'alpha_exponent': 3, 'lambda_exponent': 10, 'learning_rate': 0.04107042101489143, 'num_boost_round': 838, 'max_bin': 23, 'max_depth': 8, 'subsample': 0.47378527688070815, 'colsample_bytree': 0.6294493164087339}. Best is trial 6 with value: 0.04496761808814566.
Results for fold 0: Performance 0.05519, correlation: 0.03566, correlation with meta model: 0.52316, MMC (approx.): 0.01264
Results for fold 1: Performance 0.03820, correlation: 0.02844, correlation with meta model: 0.56586, MMC (approx.): 0.00750


Best trial: 7. Best value: 0.0466956: 100%|██████████| 8/8 [4:41:05<00:00, 2108.22s/it]

[I 2026-01-26 14:24:07,716] Trial 7 finished with value: 0.0466956224410936 and parameters: {'min_child_weight_exponent': 6, 'alpha_exponent': 1, 'lambda_exponent': 0, 'learning_rate': 0.05114360314811402, 'num_boost_round': 3597, 'max_bin': 23, 'max_depth': 4, 'subsample': 0.6893004622024659, 'colsample_bytree': 0.2779323949284377}. Best is trial 7 with value: 0.0466956224410936.


In [ ]:
# visualise result
for param in study.best_params.keys():
    optuna.visualization.plot_slice(study, params=[param]).show()

In [ ]:
save_as_pickle(study, DATA_PATH / 'results/study_xgb.pkl')